In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime
import os
import glob

In [ ]:
#subreddits = ['Minecraft','minecraftsuggestions','MinecraftDungeons','MinecraftServer']
#subreddits = ['thesims','Sims4','TheSimsBuilding','Sims3', 'thesims4']
#subreddits = ['Overwatch', 'OverwatchLeague', 'Competitiveoverwatch', 'Competitiveoverwatch']
#subreddits = ['skyrim', 'skyrimmods', 'SkyrimModsXbox', 'kyrimTogether']
#subreddits = ['roblox','RobloxDevelopers', 'RobloxAvatars', 'RobloxHelp']
#subreddits = ["GTA", "GTAOnline", "GTA6", "GrandTheftAutoV", "GTAV", "GTAV_Mods"]
#subreddits = ['leagueoflegends',  'summonerschool', 'LeagueConnect']
#subreddits = ['gaming', 'pcgaming', 'Games']
#subreddits     = ['CallOfDuty', 'CODWarzone', 'CodZombies', 'CallOfDutyMobile']
#subreddits = ['Valorant', 'ValorantCompetitive']
subreddits = ['genshin_impact', 'GenshinImpact']

name = "genshinimpact"

In [5]:
max_posts = 1000
max_scrolls = 100

# Setup ChromeDriver
service = Service(executable_path="C:/Windows/chromedriver.exe")
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
driver = webdriver.Chrome(service=service, options=options)

# Storage
all_posts = []
found_2025 = 0
seen_ids = set()

# Scrape each subreddit
for subreddit in subreddits:
    if found_2025 >= max_posts:
        break

    print(f"\nScraping r/{subreddit} for 2025 posts...")
    driver.get(f"https://www.reddit.com/r/{subreddit}/new/")
    time.sleep(10)  # Give time to load

    scroll_count = 0
    consecutive_empty_scrolls = 0
    max_empty_scrolls = 3

    while scroll_count < max_scrolls and found_2025 < max_posts:
        new_2025_posts = 0
        posts = driver.find_elements(By.TAG_NAME, "shreddit-post")

        for post in posts:
            if found_2025 >= max_posts:
                break

            post_id = post.get_attribute("id")
            if not post_id or post_id in seen_ids:
                continue
            seen_ids.add(post_id)

            title = post.get_attribute("post-title") or "No title"
            timestamp = post.get_attribute("created-timestamp")
            if not timestamp:
                continue

            try:
                dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f%z")
                if dt.year != 2025:
                    continue

                created_utc = dt.strftime('%Y-%m-%d %H:%M:%S')
                all_posts.append({
                    "subreddit": subreddit,
                    "year": dt.year,
                    "month": dt.month,
                    "text": title,
                    "created_utc": created_utc,
                })

                found_2025 += 1
                new_2025_posts += 1
                print(f"{title[:60]}")

            except Exception as e:
                print(f"Error parsing timestamp: {e}")

        scroll_count += 1
        print(f"Scroll {scroll_count} — Found {new_2025_posts} new 2025 posts — Total: {found_2025}")

        if new_2025_posts == 0:
            consecutive_empty_scrolls += 1
            print(f"No new posts found. ({consecutive_empty_scrolls} empty scrolls)")
            if consecutive_empty_scrolls >= max_empty_scrolls:
                print("Max empty scrolls reached. Moving to next subreddit.")
                break
        else:
            consecutive_empty_scrolls = 0  # reset if progress is made

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)

# Save combined posts
df = pd.DataFrame(all_posts)
filename = f"{name}-2025.csv"
df.to_csv(filename, index=False, encoding='utf-8')
print(f"\nSaved {len(df)} posts to {filename}")

driver.quit()


NameError: name 'subreddits' is not defined

In [ ]:
folder_path = "cleaned"
output_file = "2020-2025-data.csv"

csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    if not df.empty:
        dfs.append(df)
if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Merged {len(csv_files)} files into {output_file} with {len(merged_df)} rows.")
else:
    print("No files or all files are empty. Nothing was merged.")


Merged 2 files into 2020-2025-data.csv with 65900 rows.
